In [ ]:
org_name = 'mko'
org_full_name = 'Methylomonas_koyamae'

In [1]:
org_name = 'mbry'
org_full_name = 'Methylocystis_bryophila'

In [ ]:
org_name = 'mca'
org_full_name = 'Methylococcus_capsulatus'

In [2]:
import cobra
biggunimodel = cobra.io.load_json_model('other_model\\universal_model_modified.json')
#cobra.io.write_sbml_model(unimodel,"universal_model.xml")
biggunimodel

Name,bigg_universal
Memory address,1f25ea1ea90
Number of metabolites,15638
Number of reactions,28301
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [3]:
def add_to_dir(my_dir, key, value):
    if key in my_dir:
        if isinstance(my_dir[key], list):
            my_dir[key].append(value)
        else:
            my_dir[key] = [my_dir[key], value]
    else:
        my_dir[key] = value

        
KEGG_BIGG_dir = {}
for rxn in biggunimodel.reactions:
    kegganno = rxn.annotation.get('kegg.reaction')
    if kegganno:
        if type(kegganno) is list:
            for i in kegganno:
                add_to_dir(KEGG_BIGG_dir, i, rxn.id)
        else:
            add_to_dir(KEGG_BIGG_dir, kegganno, rxn.id)

EC_BIGG_dir = {}
for rxn in biggunimodel.reactions:
    ecanno = rxn.annotation.get('ec-code')
    if ecanno:
        if type(ecanno) is list:
            for i in ecanno:
                add_to_dir(EC_BIGG_dir, i, rxn.id)
        else:
            add_to_dir(EC_BIGG_dir, ecanno, rxn.id)

cyc_BIGG_dir = {}
for rxn in biggunimodel.reactions:
    cycanno = rxn.annotation.get('biocyc')
    if cycanno:
        if type(cycanno) is list:
            for i in cycanno:
                add_to_dir(cyc_BIGG_dir, i, rxn.id)
        else:
            add_to_dir(cyc_BIGG_dir, cycanno, rxn.id)

In [4]:
def printmyrxn(model, rxn):
    if type(rxn) == str:
        rxn = model.reactions.get_by_id(rxn)
    print(rxn.id,',', rxn.name)
    #print('\t https://modelseed.org/biochem/reactions/'+rxn.id.split('_')[0])
    print('\t',rxn.reaction)
    for meta, sto in rxn.metabolites.items():
        metablic = model.metabolites.get_by_id(meta.id)
        print('\t\t',metablic.id,'\t', metablic.name,'\t', sto)

In [5]:
import pandas as pd
import re

kegg_pathways = pd.read_csv('KEGG\\KEGG_pathways.txt', sep='\t',dtype={'pw_ID':str,'pw_name':str})
kegg_pathway_name = dict(zip('map'+kegg_pathways['pw_ID'].str.strip(),kegg_pathways['pw_name']))

KEGG_pathways_entrys = pd.read_csv('KEGG\\KEGG_pathways_entrys.txt', sep='\t',dtype={'pw_ID':str,'pw_name':str,'href':str,'title':str})
def findentry(entry,KEGG_pathways_entrys=KEGG_pathways_entrys,globalsearch=False):
    if globalsearch:
        search = KEGG_pathways_entrys.loc[KEGG_pathways_entrys["title"].str.contains(entry)]
    else:
        search = KEGG_pathways_entrys.iloc[17994:].loc[KEGG_pathways_entrys["title"].str.contains(entry)]     
    if len(search) == 0:
        raise TypeError('Cannot find %s!'%(entry))
    else:
        return search.reset_index()

def getPathways(entry,globalsearch=False):
    df =findentry(entry,globalsearch=globalsearch)
    pw_list = df['pw_ID'].tolist()
    return_list = list(set(pw_list))
    return_list.sort(key=pw_list.index)
    return return_list
          
def printurl(entry,globalsearch=False):
    pws = getPathways(entry,globalsearch=globalsearch)
    for pw in pws:
        print('https://www.kegg.jp/kegg-bin/show_pathway?map=%s&multi_query=%s+yellow'%(pw,entry))


def getRfromKO(entry,globalsearch=False):
    df =findentry(entry,globalsearch=globalsearch)
    R_list=[]
    for i in range(len(df)):
        string = df.loc[i,'title']
        R = re.findall(r"R\d{5}", string)
        R_list+=R
    if len(R_list) > 0:
        return_list = list(set(R_list))
        return_list.sort(key=R_list.index)
        if len(return_list) == 1:
            return return_list[0]
        else:
            return return_list
    else:
        print('Cannot find R for %s!'%(entry))
        return []

def getECfromKO(entry,globalsearch=False):
    df =findentry(entry,globalsearch=globalsearch)
    EC_list=[]
    for i in range(len(df)):
        string = df.loc[i,'title']
        elements = string.split(',')
        for element in elements:
            if len(element.split('.'))>3 and (not'(' in element):
                EC_list.append(element.strip())
    if len(EC_list) > 0:
        return_list = list(set(EC_list))
        return_list.sort(key=EC_list.index)
        if len(return_list) == 1:
            return return_list[0]
        else:
            return return_list
    else:
        print('Cannot find EC for %s!'%(entry))
        return []


In [6]:
def bigg2kegg(biggid,biggunimodel=biggunimodel):
    try:
        rxn = biggunimodel.reactions.get_by_id(biggid)
    except:
        return ''
    if rxn.annotation.get('kegg.reaction'):
        return rxn.annotation.get('kegg.reaction')
    else:
        return ''

In [7]:
def select_rxn(biggunimodel, rxnlist):
    if not isinstance(rxnlist,list):
        unirxn = biggunimodel.reactions.get_by_id(rxnlist)
        comlist=[]
        for key, value in unirxn.metabolites.items():
            comlist.append(key.id.split('_')[-1])
        if set(comlist).issubset(set(['c','e','p'])):
            return rxnlist
        else:
            return None
    else:
        newrxnlist = []
        for rxn in rxnlist:
            unirxn = biggunimodel.reactions.get_by_id(rxn)
            comlist=[]
            for key, value in unirxn.metabolites.items():
                comlist.append(key.id.split('_')[-1])
            if set(comlist).issubset(set(['c','e','p'])):
                newrxnlist.append(rxn)
        if len(newrxnlist) == 0:
            return rxnlist
        elif len(newrxnlist) > 1:
            return min(newrxnlist, key=len)
        else:
            return newrxnlist[0]
        
def kegg2bigg(keggid,biggunimodel=biggunimodel,KEGG_BIGG_dir=KEGG_BIGG_dir):
    if KEGG_BIGG_dir.get(keggid):
        selr = select_rxn(biggunimodel, KEGG_BIGG_dir.get(keggid))
        if selr:
            if type(selr) is list:
                print(keggid, selr)
                return selr
            elif type(selr) is str:
                return selr     
    else:
        return None          

In [8]:
def comparemodel(rxnlist1,rxnlist2):
    both = [r for r in rxnlist1 if r in rxnlist2]
    only1 = [r for r in rxnlist1 if r not in rxnlist2]
    only2 = [r for r in rxnlist2 if r not in rxnlist1]
    print('both: %d, only1: %d, only2: %d'%(len(both),len(only1),len(only2)))
    return [both,only1,only2]
    

In [9]:
def pathway_count(rxnlist,*args):

    if args:
        refdict={}
        for arg in args:
            refdict.update(arg.to_dict())

    pathways =[]
    for r in rxnlist:
        try:
            pathways+=getPathways(r)
        except:
            pass
    import pandas as pd
    result = pd.value_counts(pathways)
    for index, value in result.items():
        name = kegg_pathway_name.get(index)
        if args:
            if refdict.get(index):
                total_value = refdict.get(index)
            else:
                total_value = 'Inf'
            print(index, value,'/',total_value, name)
        else:
            print(index, value, name)

    return result

In [10]:
def printbypathway(rxnlist,countresult,*mapped_dir,kegg_pathway_name=kegg_pathway_name):
    refdict=countresult.to_dict()
    for map, value in refdict.items():
        name = kegg_pathway_name.get(map)
        print(map, value, name)
        
        for r in rxnlist:
            try:
                pathways = getPathways(r)
                if map in pathways:
                    if mapped_dir:
                        if mapped_dir[0].get(r):
                            print('     ',r,'\t',mapped_dir[0].get(r))
                        else:
                            print('     ',r)
                    else:
                        print('     ',r)
            except:
                pass

In [11]:
def color_compare(rxnlist1,rxnlist2):
    both = [r for r in rxnlist1 if r in rxnlist2]
    only1 = [r for r in rxnlist1 if r not in rxnlist2]
    only2 = [r for r in rxnlist2 if r not in rxnlist1]
    for r in both:
        print(r,'yellow','pink')
    for r in only1:
        print(r,'yellow','white')
    for r in only2:
        print(r,'white','pink')

In [12]:
import pandas as pd
df_kegg_reaction  = pd.read_excel(org_name+'\\'+org_name+'_clean_kegg_reaction.xlsx', index_col=0) 
keggrxnlist=df_kegg_reaction['Kegg_ID'].tolist()

R_list = []
with open(org_full_name+'_genome\\R_list.txt','r') as f:
    for line in f.readlines():
        R_list.append(line.strip())
f.close()

print('kegg_rxn_list: '+str(len(keggrxnlist)))
print('R_list: '+str(len(R_list)))

kegg_rxn_list: 980
R_list: 1055


In [13]:
kegg2bigg_list = [kegg2bigg(r) for r in keggrxnlist]

R01070 ['FBAf', 'FBAm', 'FBAh']
R00014 ['PDHam1hi', 'PDHam1mi']
R04326 ['FGFTh', 'FGFTm']


In [14]:
replace_dict ={str(['FBAf', 'FBAm', 'FBAh']):'',str(['PDHam1hi', 'PDHam1mi']):'',str(['ACCAHh', 'ACCAHm']):'',str(['FGFTh', 'FGFTm']):''}
kegg2bigg_list2 = [replace_dict[str(i)] if str(i) in replace_dict else i for i in kegg2bigg_list]
kegg2bigg_list = kegg2bigg_list2
kegg2bigg_list_removeNone = list(filter(None, kegg2bigg_list))
print('kegg2bigg_list_removeNone: '+str(len(kegg2bigg_list_removeNone)))

kegg2bigg_list_removeNone: 577


In [15]:
notmappedtoBIGG =[]
mappedtoBIGG =[]
mapped_dir = {}
for i in range(len(kegg2bigg_list)):
    mapped_dir.update({keggrxnlist[i]:kegg2bigg_list[i]})
    print(keggrxnlist[i],kegg2bigg_list[i])
    if kegg2bigg_list[i] == ''or kegg2bigg_list[i] == None:
        notmappedtoBIGG.append(keggrxnlist[i])
    else:
        mappedtoBIGG.append(keggrxnlist[i])
print('notmappedtoBIGG: '+str(len(notmappedtoBIGG)))
print('mappedtoBIGG: '+str(len(mappedtoBIGG)))

R01070 
R00710 ALDD2x
R00746 ALCD2y
R00754 ALCD2x
R00014 
R03270 None
R02569 None
R00200 PYK
R00658 ENO
R01518 PGM
R01061 GAPD
R01015 TPI
R04779 None
R02740 PGIA
R00959 PGCM
R03321 G6PI3
R01600 None
R02739 G6PI_1
R01786 GLUKA
R07618 None
R01512 PGK
R09127 None
R00341 PPCK
R00235 ACS
R00711 ALDD2y
R02073 None
R00206 PPDK
R00199 PPS
R00621 None
R03316 None
R02570 None
R00405 SUCOAS
R00268 r0082
R01899 r0422
R01082 FUM
R01900 ACONTb
R01325 ACONTa
R00351 CS
R00342 MDH
R02164 None
R01049 None
R01641 None
R01056 None
R01529 RPE
R01830 TKT2
R01528 GND
R02035 PGL
R02736 G6PBDH
R01827 TALA
R01057 None
R08572 GLYCK2
R01621 None
R10221 None
R00289 GALUi
R00286 UDPGD
R02630 None
R00867 HEX7
R01819 MPAKI
R01818 PMANM
R00888 GMAND
R00885 MAN1PT
R02568 FBA2
R03397 None
R03236 PFK_2
R00291 UDPG4E
R01105 None
R01092 GALKr
R00955 UGLT
R02957 ALDD32
R01385 UGE
R07765 None
R07764 3HAD180
R07762 None
R07763 None
R04968 None
R04726 None
R04963 None
R04960 None
R04957 None
R04952 None
R04969 None
R04543 3OAR

In [16]:
a=pathway_count(keggrxnlist)

map00230 68 Purine metabolism
map00860 51 Porphyrin metabolism
map00061 50 Fatty acid biosynthesis
map00240 37 Pyrimidine metabolism
map00790 36 Folate biosynthesis
map00270 36 Cysteine and methionine metabolism
map00680 35 Methane metabolism
map00520 34 Amino sugar and nucleotide sugar metabolism
map00620 33 Pyruvate metabolism
map00630 32 Glyoxylate and dicarboxylate metabolism
map00720 30 Carbon fixation pathways in prokaryotes
map00280 29 Valine, leucine and isoleucine degradation
map00010 28 Glycolysis / Gluconeogenesis
map00260 26 Glycine, serine and threonine metabolism
map00564 26 Glycerophospholipid metabolism
map00970 22 Aminoacyl-tRNA biosynthesis
map00330 22 Arginine and proline metabolism
map00400 22 Phenylalanine, tyrosine and tryptophan biosynthesis
map00071 21 Fatty acid degradation
map00640 21 Propanoate metabolism
map00250 21 Alanine, aspartate and glutamate metabolism
map00541 20 O-Antigen nucleotide sugar biosynthesis
map00670 20 One carbon pool by folate
map00650 1

In [17]:
b=pathway_count(notmappedtoBIGG,a)

map00061 32 / 50 Fatty acid biosynthesis
map00680 17 / 35 Methane metabolism
map00564 17 / 26 Glycerophospholipid metabolism
map00860 17 / 51 Porphyrin metabolism
map00280 15 / 29 Valine, leucine and isoleucine degradation
map00520 14 / 34 Amino sugar and nucleotide sugar metabolism
map00790 12 / 36 Folate biosynthesis
map00620 11 / 33 Pyruvate metabolism
map00550 11 / 18 Peptidoglycan biosynthesis
map00630 11 / 32 Glyoxylate and dicarboxylate metabolism
map00030 10 / 19 Pentose phosphate pathway
map00270 10 / 36 Cysteine and methionine metabolism
map00640 10 / 21 Propanoate metabolism
map00906 10 / 11 Carotenoid biosynthesis
map00380 9 / 19 Tryptophan metabolism
map00010 9 / 28 Glycolysis / Gluconeogenesis
map00230 9 / 68 Purine metabolism
map00650 9 / 19 Butanoate metabolism
map00907 9 / 9 Pinene, camphor and geraniol degradation
map00730 8 / 11 Thiamine metabolism
map00130 8 / 10 Ubiquinone and other terpenoid-quinone biosynthesis
map00780 8 / 16 Biotin metabolism
map00920 8 / 19 Su

In [18]:
c=pathway_count(mappedtoBIGG,a)

map00230 59 / 68 Purine metabolism
map00240 35 / 37 Pyrimidine metabolism
map00860 34 / 51 Porphyrin metabolism
map00270 26 / 36 Cysteine and methionine metabolism
map00720 26 / 30 Carbon fixation pathways in prokaryotes
map00790 24 / 36 Folate biosynthesis
map00260 23 / 26 Glycine, serine and threonine metabolism
map00620 22 / 33 Pyruvate metabolism
map00630 21 / 32 Glyoxylate and dicarboxylate metabolism
map00970 20 / 22 Aminoacyl-tRNA biosynthesis
map00400 20 / 22 Phenylalanine, tyrosine and tryptophan biosynthesis
map00520 20 / 34 Amino sugar and nucleotide sugar metabolism
map00250 19 / 21 Alanine, aspartate and glutamate metabolism
map00670 19 / 20 One carbon pool by folate
map00010 19 / 28 Glycolysis / Gluconeogenesis
map00680 18 / 35 Methane metabolism
map00061 18 / 50 Fatty acid biosynthesis
map00071 17 / 21 Fatty acid degradation
map00760 17 / 19 Nicotinate and nicotinamide metabolism
map00330 16 / 22 Arginine and proline metabolism
map00770 16 / 17 Pantothenate and CoA biosy

In [19]:
printbypathway(mappedtoBIGG,c,mapped_dir)

map00230 59 Purine metabolism
      R02019
      R02019 	 RNDR2
      R01072
      R01072 	 GLUPRT
      R04144
      R04144 	 PPRGL
      R04325
      R04325 	 GARFT
      R04560
      R04560 	 AICART
      R04559
      R04559 	 ADSL2r
      R04591
      R04591 	 PRASCSi
      R01127
      R01127 	 IMPC
      R00089
      R00089 	 ADNCYC
      R02017
      R02017 	 RNDR1
      R00127
      R00127 	 ADK1
      R01083
      R01083 	 ADSL1r
      R01135
      R01135 	 ADSS
      R00183
      R00183 	 NTD7
      R00190
      R00190 	 ADPT
      R02088
      R02088 	 NTD6
      R01126
      R01126 	 NTD11
      R01560
      R01560 	 ADA
      R01244
      R01244 	 ADD
      R00722
      R00722 	 NDPK9
      R01130
      R01130 	 IMPD
      R01230
      R01230 	 GMPS
      R01132
      R01132 	 HXPRT
      R01561
      R01561 	 PUNP1
      R01227
      R01227 	 NTD9
      R02142
      R02142 	 XPPT
      R02719
      R02719 	 NTD10
      R01229
      R01229 	 GPAR
      R00332
      R00332 

In [ ]:
color_compare(mappedtoBIGG,keggrxnlist)

R00710 yellow pink
R00746 yellow pink
R00754 yellow pink
R00200 yellow pink
R00658 yellow pink
R01518 yellow pink
R01061 yellow pink
R01015 yellow pink
R02740 yellow pink
R00959 yellow pink
R03321 yellow pink
R02739 yellow pink
R01786 yellow pink
R01512 yellow pink
R00341 yellow pink
R00235 yellow pink
R00711 yellow pink
R00206 yellow pink
R00199 yellow pink
R00405 yellow pink
R00268 yellow pink
R01899 yellow pink
R01082 yellow pink
R01900 yellow pink
R01325 yellow pink
R00351 yellow pink
R00342 yellow pink
R01529 yellow pink
R01830 yellow pink
R01528 yellow pink
R02035 yellow pink
R02736 yellow pink
R01827 yellow pink
R08572 yellow pink
R00289 yellow pink
R00286 yellow pink
R00867 yellow pink
R01819 yellow pink
R01818 yellow pink
R00888 yellow pink
R00885 yellow pink
R02568 yellow pink
R03236 yellow pink
R00291 yellow pink
R01092 yellow pink
R00955 yellow pink
R02957 yellow pink
R01385 yellow pink
R07764 yellow pink
R04543 yellow pink
R04566 yellow pink
R04964 yellow pink
R04534 yello